Authors: A.Radhakrishnan, GFDL
Ack: Anderson Banihirwe for intake-esm updates, GFDL colleagues for data

esgf-world.json is the ESM collections spec file for the netCDF data in the S3 bucket esgf-world. 
The catalog is updated on an on-demand basis for now. 
You can refer to https://github.com/aradhakrishnanGFDL/gfdl-aws-analysis/tree/community/esm-collection-spec-examples for the most recent catalogs
More examples can be found in https://github.com/aradhakrishnanGFDL/gfdl-aws-analysis/tree/community/examples 


In [6]:
col_url = "https://cmip6-nc.s3.us-east-2.amazonaws.com/esgf-world.json" 

In [ ]:
from glob import glob
import xarray as xr
import cftime
import numpy as np
import matplotlib.pyplot as plt
import intake, intake_esm
import pandas as pd
pd.set_option("display.max_colwidth", None)
#!pip install cmip6_preprocessing

%pip install git+https://github.com/jbusecke/cmip6_preprocessing.git


from cmip6_preprocessing.preprocessing import combined_preprocessing
from cmip6_preprocessing.preprocessing import (correct_units,rename_cmip6)

In [8]:
col = intake.open_esm_datastore(col_url)
esmcol_data = col.esmcol_data

In [9]:
def latest_version(cat):
    """
    input
    cat: esmdatastore
    output
    esmdatastore with latest DRS versions
    """
    latest_cat = cat.df.sort_values(by=['version','path']).drop_duplicates(['temporal subset','model','mip_table',
                                               'institute','variable','ensemble_member',
                                               'grid_label','experiment_id'],keep='last')
    return latest_cat

In [10]:
query_Omon_zos = dict(experiment_id=['abrupt-4xCO2'],#,'historical'],
                 mip_table=['Omon'],
                 ensemble_member=["r1i1p1f1"],
                 model=['IPSL-CM6A-LR'],#,'GFDL-ESM4'],
                 grid_label=['gn'],
                 variable=["zos"])

cat_Omon_zos = col.search(**query_Omon_zos)
cat_Omon_zos_lat = latest_version(cat_Omon_zos)


cat_Omon_zos_latest = intake.open_esm_datastore(cat_Omon_zos_lat,esmcol_data=esmcol_data)

In [11]:
cat_Omon_zos_latest.df

,project,institute,model,experiment_id,frequency,modeling_realm,mip_table,ensemble_member,grid_label,variable,temporal subset,version,path
3,CMIP6,IPSL,IPSL-CM6A-LR,abrupt-4xCO2,mon,ocean,Omon,r1i1p1f1,gn,zos,185001-214912,v20190118,s3://esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_185001-214912.nc
4,CMIP6,IPSL,IPSL-CM6A-LR,abrupt-4xCO2,mon,ocean,Omon,r1i1p1f1,gn,zos,215001-244912,v20190118,s3://esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_215001-244912.nc
5,CMIP6,IPSL,IPSL-CM6A-LR,abrupt-4xCO2,mon,ocean,Omon,r1i1p1f1,gn,zos,245001-274912,v20190118,s3://esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_245001-274912.nc


In [85]:
# Pass this function for preprocessing zos data if needed
def pp_zos(ds):
    ds = rename_cmip6(ds)
  #  ds = fix_time(ds)
    ds = correct_units(ds)
    return ds

TEST 1 - fails on ds_zos_ipsl_abrupt.sel(time=slice("2150","2151"))

In [12]:
dset_dict_zos = cat_Omon_zos_latest.to_dataset_dict(cdf_kwargs={'decode_times': False,'chunks': {'time': 1}}, storage_options={'anon':True},
                                                   )



--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


In [13]:
ds_zos_ipsl_abrupt = dset_dict_zos["CMIP6.IPSL.IPSL-CM6A-LR.abrupt-4xCO2.Omon"]


In [23]:
ds_zos_ipsl_abrupt.sel(time=slice("2150","2151"))

KeyError: '2150'

In [72]:
ds_zos_ipsl_abrupt['time']

<xarray.DataArray 'time' (time: 10800)>
array([1.550000e+01, 4.500000e+01, 7.450000e+01, ..., 1.094955e+05,
       1.095260e+05, 1.095565e+05])
Coordinates:
  * time     (time) float64 15.5 45.0 74.5 ... 1.095e+05 1.095e+05 1.096e+05
Attributes:
    axis:           T
    standard_name:  time
    long_name:      Time axis
    calendar:       gregorian
    bounds:         time_bounds

TEST 2 - turn on decodes_times and then see the time units 
also fails on ds_zos_ipsl_abrupt_t.sel(time=slice("2150","2151"))

In [20]:
dset_dict_zos = cat_Omon_zos_latest.to_dataset_dict(cdf_kwargs={'decode_times': True,'chunks': {'time': 1}}, storage_options={'anon':True},
                                                   )



--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.8/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/tim

In [24]:
ds_zos_ipsl_abrupt_t = dset_dict_zos["CMIP6.IPSL.IPSL-CM6A-LR.abrupt-4xCO2.Omon"]


In [25]:
ds_zos_ipsl_abrupt_t.sel(time=slice("2150","2151"))

TypeError: '<' not supported between instances of 'cftime._cftime.DatetimeGregorian' and 'str'

NOtice discrepancy in the following

In [73]:
ds_zos_ipsl_abrupt_t['time']

<xarray.DataArray 'time' (time: 10800)>
array([Timestamp('1850-01-16 12:00:00'), Timestamp('1850-02-15 00:00:00'),
       Timestamp('1850-03-16 12:00:00'), ...,
       cftime.DatetimeGregorian(2749, 10, 16, 12, 0, 0, 0),
       cftime.DatetimeGregorian(2749, 11, 16, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2749, 12, 16, 12, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 1850-01-16T12:00:00 ... 2749-12-16 12:00:00
Attributes:
    axis:           T
    standard_name:  time
    long_name:      Time axis
    bounds:         time_bounds

TEST 3 - works directly using xarray/fsspec on one object with open_dataset 
but fails on open_mfdataset 
ds_zos_ipsl_abrupt.sel(time=slice("2150","2151"))

In [ ]:
import xarray as xr
import s3fs

fs_s3 = s3fs.S3FileSystem()


In [81]:
#xr.show_versions() #xarray: 0.16.1

In [75]:
s3path = "s3://esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_215001-244912.nc"
ds = fs_s3.open(s3path, mode='rb')
xr_ds =xr.open_dataset(ds) #works 
xr_ds.zos.sel(time= slice("2150","2151")) 

notice time units uniformity here versus the ones that vary in the cells that fail while using intake

In [64]:
xr_ds.time 

<xarray.DataArray 'time' (time: 3600)>
array([cftime.DatetimeGregorian(2150, 1, 16, 12, 0, 0, 0),
       cftime.DatetimeGregorian(2150, 2, 15, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2150, 3, 16, 12, 0, 0, 0), ...,
       cftime.DatetimeGregorian(2449, 10, 16, 12, 0, 0, 0),
       cftime.DatetimeGregorian(2449, 11, 16, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2449, 12, 16, 12, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 2150-01-16 12:00:00 ... 2449-12-16 12:00:00
Attributes:
    axis:           T
    standard_name:  time
    long_name:      Time axis
    time_origin:    2150-01-01 00:00:00
    bounds:         time_bounds

In [86]:
import sys
dataset = []
s3path = "s3://esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_185001-214912.nc"
ds = fs_s3.open(s3path, mode='rb')
dataset.append(ds)
print(dataset)
#215001-244912
#245001-274912
index1 = 185001
index2 = 214912

while index1 <= 245001:
    index1 += 30000
    if index1 > 245001:
        break
    index2 += 30000
    s3path = 's3://esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_' + str(index1) + '-' + str(index2) + '.nc'
    ds = fs_s3.open(s3path, mode='rb')
    print(s3path)
    print(dataset)
    dataset.append(ds)
    
ds = xr.open_mfdataset(dataset, chunks={'time': 1}, combine='by_coords')

[<File-like object S3FileSystem, esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_185001-214912.nc>]
s3://esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_215001-244912.nc
[<File-like object S3FileSystem, esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_185001-214912.nc>]
s3://esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_245001-274912.nc
[<File-like object S3FileSystem, esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-4xCO2_r1i1p1f1_gn_185001-214912.nc>, <File-like object S3FileSystem, esgf-world/CMIP6/CMIP/IPSL/IPSL-CM6A-LR/abrupt-4xCO2/r1i1p1f1/Omon/zos/gn/v20190118/zos_Omon_IPSL-CM6A-LR_abrupt-

/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  
/srv/conda/envs/notebook/lib/python3.8/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, 

In [88]:
ds.zos.sel(time= slice("2749","2749"))

TypeError: '<' not supported between instances of 'cftime._cftime.DatetimeGregorian' and 'str'

In [89]:
ds['time']

<xarray.DataArray 'time' (time: 10800)>
array([Timestamp('1850-01-16 12:00:00'), Timestamp('1850-02-15 00:00:00'),
       Timestamp('1850-03-16 12:00:00'), ...,
       cftime.DatetimeGregorian(2749, 10, 16, 12, 0, 0, 0),
       cftime.DatetimeGregorian(2749, 11, 16, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2749, 12, 16, 12, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 1850-01-16T12:00:00 ... 2749-12-16 12:00:00
Attributes:
    axis:           T
    standard_name:  time
    long_name:      Time axis
    bounds:         time_bounds

TESTS repeat to make sure the errors are the same with latest intake esm too

In [ ]:
%pip install git+https://github.com/intake/intake-esm.git #ensure its latest 


In [35]:
import intake, intake_esm
col = intake.open_esm_datastore(col_url)
esmcol_data = col.esmcol_data

query_Omon_zos = dict(experiment_id=['abrupt-4xCO2'],#,'historical'],
                 mip_table=['Omon'],
                 ensemble_member=["r1i1p1f1"],
                 model=['IPSL-CM6A-LR'],#,'GFDL-ESM4'],
                 grid_label=['gn'],
                 variable=["zos"])

cat_Omon_zos = col.search(**query_Omon_zos)
cat_Omon_zos_lat = latest_version(cat_Omon_zos)


cat_Omon_zos_latest = intake.open_esm_datastore(cat_Omon_zos_lat,esmcol_data=esmcol_data)

dset_dict_zos = cat_Omon_zos_latest.to_dataset_dict(cdf_kwargs={'decode_times': True,'chunks': {'time': 1}}, storage_options={'anon':True},
                                                   )


--> The keys in the returned dictionary of datasets are constructed as follows:
	'project.institute.model.experiment_id.mip_table'


/srv/conda/envs/notebook/lib/python3.8/site-packages/numpy/core/_asarray.py:83: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return array(a, dtype, copy=False, order=order)
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.8/site-packages/xarray/coding/times.py:427: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/srv/conda/envs/notebook/lib/python3.8/site-packages/numpy/core/_asarr

In [55]:
ds_zos_ipsl_abrupt_r = dset_dict_zos["CMIP6.IPSL.IPSL-CM6A-LR.abrupt-4xCO2.Omon"]

ds_zos_ipsl_abrupt_r.time # sel(time=slice("2150","2151"))

<xarray.DataArray 'time' (time: 10800)>
array([Timestamp('1850-01-16 12:00:00'), Timestamp('1850-02-15 00:00:00'),
       Timestamp('1850-03-16 12:00:00'), ...,
       cftime.DatetimeGregorian(2749, 10, 16, 12, 0, 0, 0),
       cftime.DatetimeGregorian(2749, 11, 16, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2749, 12, 16, 12, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 1850-01-16T12:00:00 ... 2749-12-16 12:00:00
Attributes:
    axis:           T
    standard_name:  time
    long_name:      Time axis
    bounds:         time_bounds

In [58]:
ds_zos_ipsl_abrupt_r['time']#.sel(time=slice("1850","1850"))

<xarray.DataArray 'time' (time: 10800)>
array([Timestamp('1850-01-16 12:00:00'), Timestamp('1850-02-15 00:00:00'),
       Timestamp('1850-03-16 12:00:00'), ...,
       cftime.DatetimeGregorian(2749, 10, 16, 12, 0, 0, 0),
       cftime.DatetimeGregorian(2749, 11, 16, 0, 0, 0, 0),
       cftime.DatetimeGregorian(2749, 12, 16, 12, 0, 0, 0)], dtype=object)
Coordinates:
  * time     (time) object 1850-01-16T12:00:00 ... 2749-12-16 12:00:00
Attributes:
    axis:           T
    standard_name:  time
    long_name:      Time axis
    bounds:         time_bounds